In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import glob
import keras
from functools import reduce
from sklearn import preprocessing
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_time_domain_features

Using TensorFlow backend.
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tenso

In [2]:
path = r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data' # use your path
all_files = glob.glob(path + "/*.csv")

In [3]:
li=[]
frame=[]

In [ ]:
##This part is for normalizing and standardizing windows. Normalizing does not make sence because it makes all
##minimumes to 0 and all maximumes to 1


for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    df1['hr']= df1['hr'].astype(float)
   
    
    df2=df1[['hr','windowno', 'ptsd_moment']]
    hrmean=df2.groupby('windowno', as_index=False)['hr'].mean()
    hrmean=hrmean.rename(columns={"hr": "hrmean"})
    #hrmin=df2.groupby('windowno', as_index=False)['hr'].min()
    #hrmin=hrmin.rename(columns={"hr": "hrmin"})
    #hrmax=df2.groupby('windowno', as_index=False)['hr'].max()
    #hrmax=hrmax.rename(columns={"hr": "hrmax"})
    hrsd=df2.groupby('windowno').agg(np.std,ddof=0)['hr']
    hrsd=pd.DataFrame(hrsd)
    hrsd=hrsd.rename(columns={"hr": "hrsd"})
    merged= pd.merge(hrmean, hrsd, on='windowno', how='inner')
    merged= pd.merge(df2,merged, on='windowno', how='inner')
    #merged= pd.merge(hrmin,merged, on='windowno', how='inner')
    #merged= pd.merge(hrmax,merged, on='windowno', how='inner')
    stress=df2.groupby('windowno', as_index=False)['ptsd_moment'].max()
    #normalize
    #merged['nhr']=(merged['hr']-merged['hrmin'])/(merged['hrmax']-merged['hrmin'])
    merged['zhr']=(merged['hr']-merged['hrmean'])/(merged['hrsd'])
    
    
    zhrmean=merged.groupby('windowno', as_index=False)['zhr'].mean()
    zhrmean=zhrmean.rename(columns={"zhr": "zhrmean"})
    zhrmin=merged.groupby('windowno', as_index=False)['zhr'].min()
    zhrmin=zhrmin.rename(columns={"zhr": "zhrmin"})
    zhrmax=merged.groupby('windowno', as_index=False)['zhr'].max()
    zhrmax=zhrmax.rename(columns={"zhr": "zhrmax"})
    zhrsd=merged.groupby('windowno').agg(np.std,ddof=0)['zhr']
    zhrsd=pd.DataFrame(zhrsd)
    zhrsd=zhrsd.rename(columns={"zhr": "zhrsd"})
    
    
    
    
    participant = df1['participant'].iloc[0]
    data=reduce(lambda x,y: pd.merge(x,y, on='windowno', how='outer'),
       [hrmean,hrmin,hrmax,hrsd,stress])
    data['participant']= participant
    li.append(data)
frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
li=[]
framehrv=pd.DataFrame()

In [5]:
######This is the important part where you extract HRV features from HR along with HR features


for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    df1['hr']= df1['hr'].astype(float)
   
    data1=df1[['hr','windowno', 'ptsd_moment']]
    #stress=data1.groupby('windowno', as_index=False)['ptsd_moment'].max()
    data1['hrv'] = 60000/data1['hr']

    for i in data1['windowno']:
        d2=data1.loc[data1['windowno'] == i] 
        time_domain_features = get_time_domain_features(d2['hrv'])
        stress=d2['ptsd_moment'].max()
        time_domain_features.update( {'stress' : stress} )
        time_domain_features.update( {'windowno' : i} )
        li.append(time_domain_features)
    Hrfeatures=pd.DataFrame.from_dict(li)
    Hrfeatures=Hrfeatures.drop_duplicates()  
    #Hrfeatures['participant']=df1['participant'].iloc[0]
    f=[framehrv,Hrfeatures]
    framehrv=pd.concat(f)


//miniconda3/envs/PTSD/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
frame1=framehrv

In [7]:
frame1['hrrange']= frame1['max_hr']-frame1['min_hr']

In [10]:
fr=frame1.drop_duplicates()  

In [11]:
fr.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/HR Features/HR and HRV Features Correct_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [184]:
df1=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data/Windowed Participant6.csv')

In [185]:
df1['hr']= df1['hr'].astype(float)

In [186]:
df1['nhr']=(df1['hr']-df1['hr'].mean())/df1['hr'].std()

In [187]:
df2=df1[['hr','windowno', 'ptsd_moment']]

In [188]:
hrmean=df2.groupby('windowno', as_index=False)['hr'].mean()
hrmean=hrmean.rename(columns={"hr": "hrmean"})
hrmin=df2.groupby('windowno', as_index=False)['hr'].min()
hrmin=hrmin.rename(columns={"hr": "hrmin"})
hrmax=df2.groupby('windowno', as_index=False)['hr'].max()
hrmax=hrmax.rename(columns={"hr": "hrmax"})
hrsd=df1.groupby('windowno').agg(np.std,ddof=0)['hr']
hrsd=pd.DataFrame(hrsd)
hrsd=hrsd.rename(columns={"hr": "hrsd"})

In [189]:
merged= pd.merge(hrmax, hrmin, on='windowno', how='inner')
merged= pd.merge(hrsd,merged, on='windowno', how='inner')
merged= pd.merge(hrmean,merged, on='windowno', how='inner')
merged= pd.merge(df2,merged, on='windowno', how='inner')

In [190]:
merged['nhr']=(merged['hr']-merged['hrmin'])/(merged['hrmax']-merged['hrmin'])

In [191]:
merged['zhr']=(merged['hr']-merged['hrmean'])/(merged['hrsd'])

In [192]:
merged.iloc[:60]

hr  windowno  ptsd_moment     hrmean      hrsd  hrmax  hrmin       nhr  \
0   85.00       993            0  77.595833  2.418717   85.0   72.0  1.000000   
1   83.00       993            0  77.595833  2.418717   85.0   72.0  0.846154   
2   81.00       993            0  77.595833  2.418717   85.0   72.0  0.692308   
3   78.00       993            0  77.595833  2.418717   85.0   72.0  0.461538   
4   73.00       993            0  77.595833  2.418717   85.0   72.0  0.076923   
5   73.00       993            0  77.595833  2.418717   85.0   72.0  0.076923   
6   73.00       993            0  77.595833  2.418717   85.0   72.0  0.076923   
7   74.00       993            0  77.595833  2.418717   85.0   72.0  0.153846   
8   72.00       993            0  77.595833  2.418717   85.0   72.0  0.000000   
9   73.00       993            0  77.595833  2.418717   85.0   72.0  0.076923   
10  74.00       993            0  77.595833  2.418717   85.0   72.0  0.153846   
11  78.00       993            0  77.595833  2.418717   85.0   72.0  0.461538   
12  78.00       993            0  77.595833  2.418717   85.0   72.0  0.461538   
13  79.00       993            0  77.595833  2.418717   85.0   72.0  0.538462   
14  79.00       993            0  77.595833  2.418717   85.0   72.0  0.538462   
15  80.00       993            0  77.595833  2.418717   85.0   72.0  0.615385   
16  81.00       993            0  77.595833  2.418717   85.0   72.0  0.692308   
17  80.00       993            0  77.595833  2.418717   85.0   72.0  0.615385   
18  80.00       993            0  77.595833  2.418717   85.0   72.0  0.615385   
19  80.00       993            0  77.595833  2.418717   85.0   72.0  0.615385   
20  79.00       993            0  77.595833  2.418717   85.0   72.0  0.538462   
21  79.00       993            0  77.595833  2.418717   85.0   72.0  0.538462   
22  78.50       993            0  77.595833  2.418717   85.0   72.0  0.500000   
23  78.00       993            0  77.595833  2.418717   85.0   72.0  0.461538   
24  77.50       993            0  77.595833  2.418717   85.0   72.0  0.423077   
25  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
26  76.50       993            0  77.595833  2.418717   85.0   72.0  0.346154   
27  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
28  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
29  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
30  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
31  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
32  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
33  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
34  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
35  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
36  76.00       993            0  77.595833  2.418717   85.0   72.0  0.307692   
37  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
38  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
39  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
40  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
41  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
42  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
43  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
44  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
45  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
46  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
47  77.00       993            0  77.595833  2.418717   85.0   72.0  0.384615   
48  77.00       993            0  77.5

In [193]:
df2.isnull().sum()

hr             0
windowno       0
ptsd_moment    0
dtype: int64

In [194]:
zhrmean=merged.groupby('windowno', as_index=False)['zhr'].mean()
zhrmean


windowno           zhr
0         993  7.845576e-16
1         994  7.364479e-16
2         995  1.688956e-15
3         997 -3.483053e-17
4        1000 -5.286776e-17
..        ...           ...
113      3434  1.028807e-15
114      3435 -9.473903e-16
115      3486 -1.065814e-15
116      6242 -3.580469e-16
117      6352 -6.661338e-17

[118 rows x 2 columns]